In [28]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, MaxAbsScaler, RobustScaler

from GraphAPI import GraphCreator
from graph_helpers import *
from evaluations import *

%aimport GraphAPI
%aimport graph_helpers
%aimport evaluations

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [58]:
gc = GraphCreator("Sonata form", include_see_also=False)
print("Number of Links to Search:", len(gc.next_links), "\n\n")
print(list(gc.primary_nodes.keys()), "\n\n")
print(gc.see_also_articles)

Number of Links to Search: 1023 


['Introduction (music)', 'Exposition (music)', 'Musical development', 'Tonality', 'Musical analysis', 'Coda (music)', 'Classical music era', 'Concerto', 'Movement (music)', 'Sonata', 'Recapitulation (music)', 'Musical form', 'String quartet', 'Symphony'] 


['Closely related key', 'Slow movement (music)', 'Sonata rondo form']


2. We query all the nodes linked to/from the entry point (expand our network one level for each node).

In [59]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph and ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect nodes. 

In [60]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [61]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Wolfgang Amadeus Mozart,Ludwig van Beethoven,23
1,Ludwig van Beethoven,Wolfgang Amadeus Mozart,23
2,Charles-Valentin Alkan,Camille Saint-Saëns,22
3,Camille Saint-Saëns,Charles-Valentin Alkan,22
4,Camille Saint-Saëns,Claude Debussy,21


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [81]:
features_df = gc.get_features_df(rank=False)

# Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [82]:
gc.rank_similarity()
gc.features_df.sort_values("similarity_rank", ascending=False).head(10)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
505,Sonata cycle,147,2,69,78,0.097826,0.001727,0.000078,0.023116,1.0,1.0,0.096639,0,1.678371
244,Musical development,292,1,222,70,0.196210,0.006970,0.000668,0.016359,1.0,1.0,0.209302,1,1.454986
63,Sonata form,1013,1,714,299,1.000000,0.030348,0.001329,0.066148,0.0,0.0,1.000000,0,1.149778
2384,Slow movement (music),25,1,11,14,0.021635,0.000557,0.000032,0.000711,1.0,1.0,0.008345,0,0.978858
99,Tonality,646,1,492,154,0.063158,0.008716,0.001414,0.020983,1.0,1.0,0.053275,1,0.967987
608,Introduction (music),107,0,57,50,0.077388,0.001770,0.000317,0.015292,1.0,1.0,0.078322,1,0.917874
552,Arch form,127,1,70,57,0.059579,0.001527,0.000394,0.016715,1.0,1.0,0.066667,0,0.877494
520,Period (music),141,1,75,66,0.065896,0.001805,0.000518,0.016359,1.0,1.0,0.069106,0,0.864923
524,Progressive tonality,137,1,39,98,0.063927,0.002225,0.000152,0.011380,1.0,1.0,0.040055,0,0.864391
500,Developing variation,148,1,86,62,0.066514,0.001825,0.000391,0.016004,1.0,1.0,0.069519,0,0.856435


# Scaling Features

We can easily scale our each of our features through the `scale_features_df` method. It will default to `Standard Scaler`, but we can specify alternate scalers in the `scaler` parameter.  

In [83]:
scaled_feature_df = gc.scale_features_df(scaler=MinMaxScaler, copy=True) # Makes a copy of the df
scaled_feature_df.sort_values("similarity_rank", ascending=False).reset_index().drop("index", axis=1)

,node,degree,category_matches_with_source,in_edges,out_edges,shared_neighbors_with_entry_score,centrality,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry,jaccard_similarity,primary_link,similarity_rank
0,Sonata cycle,0.013396,1.0,0.006910,0.040562,0.096802,5.234363e-03,0.002911,0.118613,0.166667,0.25,0.096639,0.0,1.000000e+00
1,Musical development,0.026700,0.5,0.022231,0.036401,0.195297,2.112877e-02,0.025756,0.083942,0.166667,0.25,0.209302,1.0,8.668842e-01
2,Sonata form,0.092853,0.5,0.071500,0.155486,1.000000,9.199342e-02,0.051339,0.339416,0.000000,0.00,1.000000,0.0,6.850106e-01
3,Slow movement (music),0.002202,0.5,0.001102,0.007280,0.020524,1.687133e-03,0.001116,0.003650,0.166667,0.25,0.008345,0.0,5.831588e-01
4,Tonality,0.059180,0.5,0.049269,0.080083,0.062095,2.642216e-02,0.054638,0.107664,0.166667,0.25,0.053275,1.0,5.766809e-01
5,Introduction (music),0.009726,0.0,0.005708,0.026001,0.076341,5.364496e-03,0.012177,0.078467,0.166667,0.25,0.078322,1.0,5.468183e-01
6,Arch form,0.011561,0.5,0.007010,0.029641,0.058512,4.629100e-03,0.015129,0.085766,0.166667,0.25,0.066667,0.0,5.227558e-01
7,Period (music),0.012845,0.5,0.007511,0.034321,0.064836,5.472193e-03,0.019929,0.083942,0.166667,0.25,0.069106,0.0,5.152645e-01
8,Progressive tonality,0.012478,0.5,0.003905,0.050962,0.062864,6.744622e-03,0.005762,0.058394,0.166667,0.25,0.040055,0.0,5.149478e-01
9,Developing variation,0.013487,0.5,0.008612,0.032241,0.065454,5.532096e-03,0.015018,0.082117,0.166667,0.25,0.069519,0.0,5.102069e-01


___
# Validation

Here, we _validate_ our results. For many articles, we already have some user defined links that are highly related to the present article. These are found in the **See Also** section of several Wikipedia articles (some pages do not have them). These are not ordered in importance in any meaningful way, and there are no rating scores.

The intuition in this validation is as follows: 

> _Given that we know some articles are highly related from user input, if the recommendations provided by this system are valid, we would expect to see those **See Also** links ranked relatively high on our list._ 

_Note: This validation is not meant as **confirmation** or **evaluation** of the results. It only provides us one way of telling if the results we are seeing are reasonably valid. It is important to note that we cannot compare these results across two different articles, as those would be two entirely different network structures, likely with different human labeled links._  


In [84]:
evaluate_metrics(scaled_feature_df, 
                 on=["similarity_rank", "centrality", "adjusted_reciprocity", "page_rank", "shortest_path_length_from_entry", "jaccard_similarity"], 
                 targets=gc.see_also_articles)

Metric Score,score,max score possible,difference,total targets,% targets in top 1%,% targets in top 5%,% targets in top 10%,% targets in top 20%
similarity_rank,0.993124,0.999967,0.006843,3.0,1.000000,1.000000,1.0,1.0
centrality,0.927167,0.999967,0.072800,3.0,0.333333,0.333333,1.0,1.0
adjusted_reciprocity,0.935401,0.999967,0.064567,3.0,0.666667,0.666667,1.0,1.0
page_rank,0.966912,0.999967,0.033055,3.0,0.666667,1.000000,1.0,1.0
shortest_path_length_from_entry,0.000011,0.999967,0.999956,3.0,0.000000,0.000000,0.0,0.0
jaccard_similarity,0.960047,0.999967,0.039920,3.0,0.333333,1.000000,1.0,1.0


The chart generated above compares different ranking metrics (left index) for a given article. The most important column, `score`, provides a fast way for us to compare these different metrics. 

For example, if we see a _score_ of 0.98 for a given ranking metric, The following statement would be true:

> All of the human labeled **See Also** links are present within the top 98% of our recommendations. 

Since the human labeled links comprise a range, it is not possible to get a score of 100%. The `max score possible` column indicates the score that would be achieved if all the human labeled _See Also_ links appeared at the top of our recommendations without any other links intervening.   

The `difference` column is an alternative way of looking at the score. If we had a 0.02 in this column, we could say:

> All the human labeled **See Also** links are contained within the top 2% of our recommendations. 

`Total targets` is the number of human labeled _See Also_ links. 

Because it is possible that different metrics could have similar scores, we want a way to break down the dispersion of the known related links to see if one metric does perform better than another. The trailing four columns provide us with a course way of measuring this dispersion. 

Each of these columns indicates the percentage of human labeled _See Also_ links captured within a given percentage of the top of our recommendations. For example, if we see a 0.92 in the `% targets in the top 1%` columns, we could say:

> 92% of the human labeled **See Also** links appear in the top 1% of our recommendations. 

The value of these columns is a follows - If two ranking metrics have similar scores, we _might_ consider the better performing one to be the one in which the majority of the human labeled links are higher in our recommendation list. 

___

In [66]:
gc.features_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91335 entries, 0 to 91334
Data columns (total 14 columns):
node                                 91335 non-null object
degree                               91335 non-null float64
category_matches_with_source         91335 non-null float64
in_edges                             91335 non-null float64
out_edges                            91335 non-null float64
shared_neighbors_with_entry_score    91335 non-null float64
centrality                           91335 non-null float64
page_rank                            91335 non-null float64
adjusted_reciprocity                 91335 non-null float64
shortest_path_length_from_entry      91335 non-null float64
shortest_path_length_to_entry        91335 non-null float64
jaccard_similarity                   91335 non-null float64
primary_link                         91335 non-null float64
similarity_rank                      91335 non-null float64
dtypes: float64(13), object(1)
memory usage: 9.8+ MB

In [ ]:
df = gc.features_df.sort_values("similarity_rank", ascending=False).reset_index().drop("index", axis=1)
see_also_indeces = []
for node in gc.see_also_articles:
    article = df[df.node == node]
    
    see_also_indeces.append((node, article.index[0]))
    
see_also_indeces

In [ ]:
len(gc.graph.nodes)